# Process AGAGE JFJ and CGO data for Johannes' gases.

In [1]:
import pandas as pd
from py12box_invert import utils
from pathlib import Path
from py12box_agage import agage_process 
import getpass
from datetime import datetime
import numpy as np

/user/home/lw13938/work/py12box_agage/py12box_agage/agage_process.py:39: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if site is not "CGO":


In [19]:
species = "CFC-115" #, -115, and -114
output_directory=None
repeatability=0.

In [20]:
if species != "CFC-115":
    sites =  {"JFJ":0, "MHD":0, "THD":0, "RPB":1, "SMO":2, "CGO":3}
else:
    sites =  {"JFJ":0, "MHD":0, "THD":0, "SMO":2, "CGO":3}

# Check if species is in data selector
data_selector = pd.read_csv("/user/home/lw13938/work/py12box_agage/py12box_agage/agage_data_selector.csv",
                            index_col="Species")
    
if species not in data_selector.index:
    print(f"Exiting: {species} not in agage_data_selector.csv")


# Array of sites and 12-box model box numbers
species_in = species

dfs = []
rmsite = []
comment_string = f"# AGAGE data for {species}\n"
comment_string += f"# Created by {getpass.getuser()} on {datetime.now()}\n"
comment_string += f"# Contact AGAGE data owners before use \n"
comment_string += f"# This file contains data from the following sites/instruments: \n"
comment_string += "#===================================================\n"

for site in sites.keys():
    
    df_site, comment_site = agage_process.combine_instruments(species, site)

    if df_site is not None:
        dfs.append(df_site)
        comment_string += comment_site
    else:
        rmsite = rmsite + [site]

for rms in rmsite:
    sites.pop(rms)

if len(dfs) == 0:
    print("No data found")

for bi in range(4):
    if bi not in sites.values():
        df_empty = dfs[0].copy() #
        df_empty.loc[:] = np.nan
        dfs.append(df_empty)
        sites[f"XX{bi}"] = bi

# Find units
units = [df.xs("mf", level="var", axis=1).columns.get_level_values("unit").values[0] for df in dfs]
if len(set(units)) != 1:
    raise(f"ERROR: Units don't match: {units}")

# Find scales
scales = [df.xs("mf", level="var", axis=1).columns.get_level_values("scale").values[0] for df in dfs]
if len(set(scales)) != 1:
    raise(f"ERROR: Units don't match: {scales}")

# Concatenate and average numeric data
df_concat = pd.concat([df.xs(units[0], level="unit", axis=1) for df in dfs], 
                      axis=1, keys=[(site, box) for site, box in sites.items()], names=["site", "box"])
df_grouped = df_concat.groupby(by=["var", "box", "scale"], axis=1).mean()

# Drop scale level
df_grouped = df_grouped.droplevel("scale", axis=1)

# Collect instrument data
df_instrument_concat = pd.concat([df.xs("instrument", level="var", axis=1) for df in dfs], 
                                  axis=1, keys=[(site, box) for site, box in sites.items()], names=["site", "box"])


instrument_list = pd.DataFrame(
                         columns=pd.MultiIndex.from_tuples((("instruments", 0),
                                                            ("instruments", 1),
                                                            ("instruments", 2),
                                                            ("instruments", 3)), names=["var", "box"]),
                         index=df_grouped.index)#, dtype=pd.StringDtype())

for bi in range(4):
    df_instrument_concat_bi = df_instrument_concat.xs(bi, level="box", axis=1)
    instrument_list.iloc[:, bi] = df_instrument_concat_bi.iloc[:, 0]
    for sitei in range(len(df_instrument_concat_bi.columns))[1:]:
        instrument_list.iloc[:, bi] = instrument_list.iloc[:, bi].str.cat(df_instrument_concat_bi.iloc[:,sitei],
                                                                            sep="|",
                                                                            na_rep="")

# Collect sites
site_list = pd.DataFrame(
                         columns=pd.MultiIndex.from_tuples((("sites", 0),
                                                            ("sites", 1),
                                                            ("sites", 2),
                                                            ("sites", 3)), names=["var", "box"]),
                         index=df_grouped.index)#, dtype=pd.StringDtype())

for i in df_concat.index:
    dfi = df_concat.loc[i].dropna()
    if len(dfi) == 0:
        site_list.loc[i] = ["", "", "", ""]
    else:
        site_row = []
        for bi in range(4):
            if bi in dfi.index.get_level_values("box"):
                site_set = set(dfi.xs(bi, level="box").index.get_level_values("site"))
                if len(site_set) == 0:
                    site_row.append("")
                else:
                    site_row.append("|".join(site_set))
            else:
                site_row.append("")
        site_list.loc[i] = site_row

# Put measurements, sites and instruments together into one dataframe
df_grouped = pd.concat([df_grouped, site_list, instrument_list], axis=1)

# Modify uncertainties
#  at this stage, uncertainties are either standard deviations of repeat samples (archive) or variability (AGAGE)
############################################

# For archive data, add representation error
for bi in range(4):

    if bi==0 or bi == 1 or bi == 2:
        continue

    df_variability = df_grouped[("mf_variability", bi)].copy()

    wh_archive = df_grouped[("instruments", bi)].str.contains("Archive", na=False)
    wh_insitu = np.isfinite(df_variability) & ~wh_archive

    mf_insitu_av = (df_variability[wh_insitu]).mean()
    mf_insitu_variability_av = (df_variability[wh_insitu]).mean()

    df_variability[wh_archive] = \
        np.sqrt(df_variability[wh_archive]**2 + \
            (df_variability[wh_archive] / mf_insitu_av * mf_insitu_variability_av)**2)

    df_grouped[("mf_variability", bi)] = df_variability.copy()

# Add a repeatability
df_grouped["mf_variability"] = np.sqrt(df_grouped["mf_variability"]**2 + (df_grouped["mf"]*repeatability)**2)

Reading /user/home/lw13938/shared/obs_raw/AGAGE_GATech/gcms/monthly_data/JFJ-medusa.mon
Reading /user/home/lw13938/shared/obs_raw/AGAGE_GATech/gcms/monthly_data/MHD-medusa.mon
Reading /user/home/lw13938/shared/obs_raw/AGAGE_GATech/ale_gage_agage/monthly_data/MHD-agA.mon
Reading /user/home/lw13938/shared/obs_raw/AGAGE_GATech/gcms/monthly_data/THD-medusa.mon
Reading /user/home/lw13938/shared/obs_raw/AGAGE_GATech/ale_gage_agage/monthly_data/THD-agA.mon
Reading /user/home/lw13938/shared/obs_raw/AGAGE_GATech/gcms/monthly_data/SMO-medusa.mon
Reading /user/home/lw13938/shared/obs_raw/AGAGE_GATech/ale_gage_agage/monthly_data/SMO-agA.mon
Reading /user/home/lw13938/shared/obs_raw/AGAGE_GATech/gcms/monthly_data/CGO-medusa.mon
Reading /user/home/lw13938/shared/obs_raw/AGAGE_GATech/ale_gage_agage/monthly_data/CGO-agA.mon
Reading /user/home/lw13938/shared/obs_raw/AGAGE_archive/output/CFC115/CFC115_archive_nonan.csv


In [21]:
# Append site and units
comment_string += f"# SCALE: {scales[0]}\n# UNITS: {units[0]}\n"

# If no output directory given, default to data folder
if not output_directory:
    output_directory = f"/user/home/lw13938/work/py12box_laube/data_allagage/{species}/inputs/"
# If output directories don't exist, create them
if not Path(output_directory).exists():
    if not Path(output_directory).parent.exists():
        print(f"... creating directory {Path(output_directory).parent}")
        Path(output_directory).parent.mkdir()
    print(f"... creating directory {output_directory}")
    Path(output_directory).mkdir()
    
with open(Path(output_directory) / f"{species}_obs_agage.csv", 'w') as fout:
    fout.write(comment_string)
    df_grouped.to_csv(fout)

False

'Archive'